## MNIST classification

In this notebook we tackle the perhaps most well known problem in all of machine learning, classifying hand-written digits.

The particular dataset we will use is the MNIST (Modified National Institute of Standards and Technology)
The digits are 28x28 pixel images that look somewhat like this:

![](https://user-images.githubusercontent.com/2202312/32365318-b0ccc44a-c079-11e7-8fb1-6b1566c0bdc4.png)

Each digit has been hand classified, e.g. for the above 9-7-0-9-0-...

Our task is to teach a machine to perform this classification, i.e. we want to find a function $\mathcal{T}_\theta$ such that

| | |
|-|-|
|$\mathcal{T}_\theta$(|<img align="center" src="https://user-images.githubusercontent.com/2202312/33177374-b134e572-d062-11e7-87c7-0574c6f5bee9.png" width="28"/>|) = 4|

# Import dependencies

This should run without errors if all dependencies are installed properly.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# Tensorflow

In these exercies we'll use the software library [tensorflow](https://www.tensorflow.org/), which is a state of the art library for numerical computations in python developed by Google. The package supports computation on both the CPU and GPU, and also supports automatic differentiation (more on that later).

Writing code in tensorflow is quite similar to classical python, except for one thing: tensorflow uses *deffered execution*, this means that instead of executing the code in one step (called eager execution), the computational graph is first defined, and it is then fed data upon which the computation is performed. Users from compiled languages will recongnize this execution method.

The execution flow typically goes like this:

* Define placeholders for the input
* Define what computations should be performed
* Feed the placeholders data

This computational model has several upsides that are sadly not evident in simple samples like these, including parallelization, packaging and optimization.

In [2]:
# Start a tensorflow session
session = tf.InteractiveSession()

# Set the random seed to enable reproducible code
np.random.seed(0)

# Get data and utilities

We now need to get the data we will use, which in this case is the famous [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, a set of digits 70000 hand-written digits, of which 60000 are used for training and 10000 for testing.

In addition to this, we create a utility `evaluate(...)` that we will use to evaluate how good the classification is.

### Get MNIST data

In [3]:
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
import matplotlib.cm

In [5]:
plt.imshow(mnist.test.images[0].reshape(28,-1),cmap='Greys_r')

Read the 10000 mnist test points

In [6]:
batch = mnist.test.next_batch(10000)
test_images = batch[0].reshape([-1, 28, 28, 1])
test_labels = batch[1]

def evaluate(result_tensor, data_placeholder):
    """Evaluate a reconstruction method.

    Parameters
    ----------
    result_tensor : `tf.Tensor`, shape (None,)
        The tensorflow tensor containing the result of the classification.
    data_placeholder : `tf.Tensor`, shape (None, 28, 28, 1) or (None, 784)
        The tensorflow tensor containing the input to the classification operator.

    Returns
    -------
    MSE : float
        Mean squared error of the reconstruction.
    """
    feed_images = np.reshape(test_images, [-1, *data_placeholder.shape[1:]])
    result = result_tensor.eval(
        feed_dict={data_placeholder: feed_images})

    return np.mean(result == test_labels)

# Logistic regression


We start with [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression), perhaps the most well known and widely applied classification method.

The first problem we need to solve is that the values we try to regress against are discrete (e.g. [0, 1, 2, ..., 9]) which does not work very well with continuous optimization. To solve this we convert the values to a one-hot encoding, embedding the values into $\mathbb{R}^{10}$:



In [7]:
toh = tf.one_hot([0, 1, 2], depth=3)
toh.eval()

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]], dtype=float32)

This can be seen as a probabilistic encoding, i.e. we can estimate that a number is 10% 1 and 90% 2. For our training data, we have 100% certanity for each digit. 

The estimator used for logistic regression is

$$
p_x(\text{label=$i$}) = \frac{\exp(\langle w_i, x \rangle + b_i)}{\sum_{j=0}^9 \exp(\langle w_j, x \rangle + b_j)}
$$

Here, $p_x(\text{label=$i$})$ is the probability of an image $x$ belonging to a category $i$, $w_i \in \mathbb{R}^{28 \times 28}$ and $b_i \in \mathbb{R}$

The loss function is a comparison between the probability distribution $p_x$ and the deterministic probability distribution $q_x$.
We use the *cross entropy* for this. The loss function for each image is:
\\[
-\sum_{i=0}^9 q_x(\text{label=$i$}) \ln(p_x(\text{label=$i$}))
\\]

The final loss function is the mean value of the cross entropy (implicitly assuming that the images are uniformly distributed):
\\[
L(p) := -\frac{1}{N}\sum_{x=1}^N\sum_{i=0}^9 q_x(\text{label=$i$})\ln(p_x(\text{label=$i$}))
\\]

### Elementary Implementation

We start with an elementary implementation in `TensorFlow`.

In [8]:
with tf.name_scope('elementary_network'):
    # Create a placeholder for our input data (no computation is done here)
    X = tf.placeholder(shape=(None, 784), dtype=tf.float32, name="X")
    
    # Create the parameters (weight, bias) of the model
    weights = tf.Variable(tf.random_normal((784, 10)), name="weights")
    bias = tf.Variable(tf.zeros((10)), name="bias")
    
    # Compute the probabilities (this is all lazy, no computations are actually performed)
    lin = tf.matmul(X, weights) + bias
    elin = tf.exp(lin)
    Z = tf.reduce_sum(elin, axis=1, keep_dims=True)
    prob = elin / Z
    log_prob = tf.log(prob)

Define the loss function which measures how good our parameters are

In [9]:
with tf.name_scope("elementary_loss"):
    labels = tf.placeholder(shape=(None,), dtype=tf.int32)
    determ = tf.one_hot(labels, depth=10)
    loss = -tf.reduce_mean(determ*log_prob)

### Gradient descent

Define the gradient descent update step, i.e.

$$w_i \leftarrow w_i - \omega \nabla_{w_i} L(w, b)$$

where $\omega$ is the *learning rate*, or step size.

Note that in machine learning, we typically use *stochastic* gradient descent (SGD). In these methods we don't use all of the data to compute the gradient, only a small subset called a mini-batch. Here we use 128 images in each training step.

Further, while for this case computing the gradient would be quite simple, once we move to harder and mroe complicated models doing so would be basically impossible to do by hand. To work around this, all major deep learning frameworks implement [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation). This may sound fancy, but automatic differentiation is simply the chain rule for the derivative. Tensorflow implements it using the `tf.gradients` command.

In [10]:
with tf.name_scope("elementary_training"):
    learning_rate = .1
    batch_size = 128

    variables = [weights, bias]
    gradients = tf.gradients(loss, variables)
    update_ops = [var.assign(var - learning_rate*grad) 
                  for var, grad in zip(variables, gradients)]

Since all the code above was lazy, nothing has actually happened. Before we start we need to initialize the variables

In [11]:
init = tf.global_variables_initializer().run()

We train the network by feeding data from the training set and occationally evalute the performance on our test set, this is the first point we actually start doing computations

In [12]:
for i in range(100000):
    images_, labels_ = mnist.train.next_batch(batch_size)
    session.run(update_ops, 
                feed_dict={labels:labels_, X:images_})
    
    if i % 1000 == 0:
        print("{:.1f}%, ".format(evaluate(tf.argmax(log_prob, axis=1), X)*100), end="")

7.4%, 40.8%, 58.0%, 66.2%, 70.9%, 73.8%, 75.7%, 77.3%, 78.5%, 79.5%, 80.2%, 80.7%, 81.3%, 81.9%, 82.5%, 82.8%, 83.2%, 83.5%, 83.8%, 84.1%, 84.3%, 84.5%, 84.8%, 85.0%, 85.1%, 85.4%, 85.5%, 85.5%, 85.8%, 86.0%, 86.1%, 86.2%, 86.3%, 86.4%, 86.6%, 86.7%, 86.8%, 86.8%, 86.9%, 86.9%, 87.1%, 87.1%, 87.2%, 87.4%, 87.4%, 87.5%, 87.6%, 87.6%, 87.7%, 87.8%, 87.8%, 87.8%, 87.9%, 88.0%, 88.0%, 88.1%, 88.3%, 88.3%, 88.3%, 88.3%, 88.3%, 88.4%, 88.5%, 88.5%, 88.5%, 88.5%, 88.6%, 88.6%, 88.6%, 88.6%, 88.6%, 88.7%, 88.7%, 88.8%, 88.9%, 88.8%, 88.8%, 88.9%, 88.9%, 88.9%, 89.0%, 89.0%, 89.0%, 89.1%, 89.1%, 89.2%, 89.2%, 89.2%, 89.2%, 89.3%, 89.3%, 89.3%, 89.4%, 89.4%, 89.4%, 89.3%, 89.5%, 89.5%, 89.6%, 89.6%, 

## Using TensorFlow libraries

While the above code solves our problem, it involved several small and perhaps obscure steps. Once we start moving to more complicated neural networks the code would become very repetetive.

Since all of the steps are standardized, we can (and should) instead use built in tensorflow functions, this example does that, and all following examples will do the same.

### Placeholders

In [13]:
with tf.name_scope('placeholders'):
    images = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    true_labels = tf.placeholder(tf.int32, shape=[None])

### Network

The "network" can be computed using the `tf.contrib.layers.fully_connected` function, which computes

$$\rho(Ax + b)$$

where $\rho$ is the activation function, $A$ the weights and $b$ the bias. Note that here we never explicitly construct these, they are hidden inside tensorflow.

In [14]:
with tf.name_scope('logistic_regression'):
    x = tf.contrib.layers.flatten(images)
    logits = tf.contrib.layers.fully_connected(x, 
                                               num_outputs=10,
                                               activation_fn=None)

### Loss and optimization

The loss function defined above should be done using the `tf.nn.softmax_cross_entropy_with_logits` function, which not only is easier to use, it is also more numerically stable

In [15]:
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

In [16]:
session.run(tf.global_variables_initializer())

### Train the network

Training the network looks about the same as above

In [17]:
# Initialize all TF variables
for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 1000 == 0:
        print("{:.1f}%, ".format(evaluate(tf.argmax(logits, axis=1), images)*100), end="")

12.2%, 91.3%, 92.1%, 92.5%, 92.5%, 92.7%, 92.8%, 92.8%, 92.9%, 92.7%, 

# Multilayer Perceptron

The first "deep" neural networks were [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron), in these we have a function of the following form

$$
\rho(W_3\rho(W_2\rho(W_1 x + b_1) + b_2) + b_3)
$$

Where $W_i$ are matrices and $b_i$ vectors. Note that the logistic regression can be cast into this form (how?).

In [18]:
with tf.name_scope('logistic_regression'):
    x = tf.contrib.layers.flatten(images)
    x = tf.contrib.layers.fully_connected(x, num_outputs=128)  # the default activation function is ReLU
    x = tf.contrib.layers.fully_connected(x, num_outputs=32)
    logits = tf.contrib.layers.fully_connected(x, 
                                               num_outputs=10,
                                               activation_fn=None)
    pred = tf.argmax(logits, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 1000 == 0:
        print("{:.1f}%, ".format(evaluate(tf.argmax(logits, axis=1), images)*100), end="")

10.0%, 96.4%, 97.4%, 

KeyboardInterrupt: 

# Convolutional network

Convolutional neural networks are a corner-stone of the deep learning revolution. Here instead of using traditionall fully-connected layers which connect each point with all other points, we use spatial convolutions instead. By doing this, we get a translation invariant operator that acts locally. In order to get non-local behaviour we stack several of these on top of each other.

The following code is a very simplified convolutional neural network for digit classification:

In [ ]:
with tf.name_scope('convolutional_network'):
    x = tf.contrib.layers.conv2d(images, 
                                 num_outputs=32, # Number of "channels", e.g. duplicates of the image
                                 kernel_size=3,  # size of the convolution kernel
                                 stride=2)       # Use strides (jumps) to decrease the image size in each step
    x = tf.contrib.layers.conv2d(x, num_outputs=32, kernel_size=3, stride=2)
    x = tf.contrib.layers.flatten(x)
    
    x = tf.contrib.layers.fully_connected(x, 128)
    logits = tf.contrib.layers.fully_connected(x, 10,
                                               activation_fn=None)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 1000 == 0:
        print("{:.1f}%, ".format(evaluate(tf.argmax(logits, axis=1), images)*100), end="")

# Conclusions

* Software libraries like tensorflow are great
* Neural networks can be used to classify numbers
* Deeper neural networks seem to work quite well